**Running the Code**
1.   Following is the implementation of various heuristics for set covering problem with A* algorithm.
2.   to test the algorithm with different heuristic substitue the h_function in the cost function with the desired heuristic.



In [ ]:
from random import random
from functools import reduce
import numpy as np
from queue import PriorityQueue
from collections import namedtuple

PROBLEM_SIZE = 15
NUM_SETS = 15
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)

State = namedtuple('State', ['taken', 'not_taken'])


def goal_check(state):
    return np.all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))


def distance(state):
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )


def g_function(state):
    return len(state.taken)


def h_function1(state):
    return distance(state)

def h_function2(state):
    return len(state.not_taken)

def h_function_min_set_coverage(state):
    uncovered = reduce(
        np.logical_or,
        [SETS[i] for i in state.not_taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )
    min_sets = sum(uncovered) // PROBLEM_SIZE + 1
    return min_sets


def calculate_weights(SETS):
    num_sets = len(SETS)
    element_sums = np.sum(SETS, axis=0)  # Sum along the columns to get the sum of occurrences of each element
    weights = element_sums / num_sets    # Normalize by the total number of sets
    return weights

def h_function_weighted(state):
    uncovered = reduce(
        np.logical_or,
        [SETS[i] for i in state.not_taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )
    WEIGHTS=calculate_weights(SETS)
    weighted_distance = np.sum(uncovered * WEIGHTS)
    return weighted_distance

def costFunction(state):
    return g_function(state) + h_function_weighted(state)


assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Problem not solvable"

def optimizer():
    frontier = PriorityQueue()
    start_state = State(set(), set(range(NUM_SETS)))
    frontier.put((costFunction(start_state), start_state))
    visited_states = set()


    counter=0
    while not frontier.empty():
        counter += 1
        _, current_state = frontier.get()

        # Convert the current_state to a frozenset before adding it to visited_states
        current_state_frozen = (frozenset(current_state.taken), frozenset(current_state.not_taken))

        if current_state_frozen not in visited_states:
            visited_states.add(current_state_frozen)

            if goal_check(current_state):
                break

            for action in current_state.not_taken:
                new_state = State(
                    current_state.taken | {action},
                    current_state.not_taken - {action},
                )

                # Convert the new_state to a frozenset before checking for membership
                new_state_frozen = (frozenset(new_state.taken), frozenset(new_state.not_taken))
                if new_state_frozen not in visited_states:
                    frontier.put((costFunction(new_state), new_state))
    return counter

counter = optimizer()
print(counter)
#print(f"Solved in {counter:,} steps ({len(current_state.taken)} sets selected)")
#print("Selected sets:", current_state.taken)